# Monte Carlo Markov Chain

Code adapted from [MCMC Intuition for Everyone](https://towardsdatascience.com/mcmc-intuition-for-everyone-5ae79fff22b1) by Rahul Agarwal.

In [1]:
import numpy as np

Q = np.matrix([[0.9  , 0.075, 0.025],
               [0.15 , 0.80 , 0.05 ],
               [0.25 , 0.25 , 0.5  ]])

In [2]:
# state 1
init_s = np.matrix([[1, 0, 0]])
np.dot(init_s, Q)

matrix([[0.9  , 0.075, 0.025]])

In [3]:
def find_stationary(init_s, Q, tol=10e-9):
    """
    Find a stationary state in a Markov Chain.
    
    Args
    ----
        init_s : initial state
        Q : matrix of transition probabilities
        tol : a tolerance limit
    """
    
    epsilon = 1
    while epsilon > tol:
        next_s = np.dot(init_s, Q)
        epsilon = np.sqrt(np.sum(np.square(next_s - init_s)))
        init_s = next_s
    
    # test equality
    try:
        assert init_s.all() == next_s.all()
    except AssertionError:
        print('stationary state not found.')
        
    return next_s

In [4]:
stat_s1 = find_stationary(init_s, Q)
stat_s1

matrix([[0.62500002, 0.31249998, 0.0625    ]])

In [5]:
# state 2
init_s = np.matrix([[0, 1, 0]])
stat_s2 = find_stationary(init_s, Q)
stat_s2

matrix([[0.62499998, 0.31250002, 0.0625    ]])

In [6]:
# state 3
init_s = np.matrix([[0, 0, 1]])
stat_s3 = find_stationary(init_s, Q)
stat_s3

matrix([[0.62499998, 0.31250002, 0.0625    ]])

> "The stationary state distribution is important because it lets you define the probability for every state of a system at a random time."